<a href="https://colab.research.google.com/github/Shubhankar9934/Deep_Learning/blob/main/FakeNewsClassifierUsingBiLSTM(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake News Classifier Using LSTM**

Dataset: https://www.kaggle.com/c/fake-news/data

In [1]:
#Install the Kaggle library
!pip install kaggle
# Make a directory named “.kaggle”
! mkdir ~/.kaggle
# Copy the “kaggle.json” into this new directory
! cp kaggle.json ~/.kaggle/
# Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! kaggle competitions download -c fake-news

 71% 33.0M/46.5M [00:01<00:00, 21.2MB/s]
100% 46.5M/46.5M [00:01<00:00, 25.8MB/s]


In [3]:
! unzip fake-news

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [7]:
###Drop Nan Values
df=df.dropna()

In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [11]:
y=df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [12]:
X.shape

(18285, 4)

In [13]:
y.shape

(18285,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.8.2'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Whenever you are giving input to the embeding layer make sure that we have 
# to keep our Size input length fixed that's why we need to use pad on bodh side 
# either on 'pre' side or 'post' side.
# we Add '0' Padded on pre or post so that we make the sentences equal in length.
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
# one_hot helps us to convert the sentences over given vocabulary size
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
# Since this is a classification problem so that my final layer
# must have one node so that it identify the news is fake or not.

**One_Hot Encoading**

In [17]:
### Vocabulary size
voc_size=5000

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]


'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages.reset_index(inplace=True)
# Reason of doing reset_index because we drop NaN value from this data set.

In [21]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


**Data Preprocessing using nltk**

In [22]:
import nltk # importing Natural Language Toolkit
import re   # re stands for regular Expression
from nltk.corpus import stopwords #(stopwords:words that you do not want to use to describe the topic of your content.)

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Why we need preprocessing?

Because we need to clean the data, data may be having various special character,it will having spaces,it will having words that are not important for that we can remove it by using stopwords.

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
# PorterStemmer : we need PorterStemmer because we need to Stemming Sentences
# which means sees the entire sentence as a word, so it returns it as it is.
ps = PorterStemmer()# Intialization of PorterStemmer.
corpus = [] # we are creating a list.
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # using regular-expression
    # and substituting apart from this->'[^a-zA-Z]' character and substituting
    # everythings with blank. simply means skipping the character from the sentences.
    review = review.lower()
    #lowering the words in that sentences reason is that if any capital letter 
    # thanks and Thanks to make both are same, if we do not lower them that would 
    # be treated as two different words.
    review = review.split()
    # we are using split because we need to apply stopwords.
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

**One Hot Representation**

In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] # for words in corpus 
# we are appying onehot encoding by passing words and vocabulary size.
onehot_repr

[[830, 4277, 4052, 9, 1874, 1493, 2622, 1782, 1608, 4535],
 [1966, 2742, 2049, 3752, 2902, 186, 4512],
 [4264, 2302, 4953, 1099],
 [3261, 3437, 275, 726, 4597, 976],
 [1970, 2902, 586, 3099, 1893, 1223, 2902, 776, 1748, 1562],
 [4798,
  4355,
  2997,
  195,
  3831,
  4548,
  2128,
  4778,
  88,
  4662,
  1030,
  2572,
  4975,
  1763,
  4512],
 [1407, 2255, 4344, 81, 314, 2957, 3546, 3477, 3204, 4837, 462],
 [3889, 3922, 869, 4963, 2575, 1192, 4548, 4028, 3204, 4837, 462],
 [4793, 4436, 2522, 2420, 29, 226, 2429, 3903, 4548, 3854],
 [370, 647, 955, 2684, 4790, 3168, 4516, 4894],
 [1633, 1718, 4059, 3357, 2306, 2840, 470, 4450, 2448, 2759, 789],
 [726, 2678, 1874, 226, 4548, 2575],
 [2533, 4304, 2697, 3772, 2060, 1548, 478, 3269, 3938],
 [3614, 572, 3953, 2627, 4344, 2993, 182, 3204, 4837, 462],
 [4767, 3092, 3470, 2750, 697, 3204, 4837, 462],
 [935, 141, 4746, 1454, 3013, 3445, 4231, 2614, 2904, 3951],
 [590, 936, 2742],
 [1830, 1530, 558, 2844, 4548, 1339, 1667, 4512],
 [2339, 2295, 20

**Embedding Representation**

In [30]:
# In Embedded Representation what we are doing is that we are saying
# make all the sentences of a fixed length.

# In order to make it possible we are doing pad_Sequences
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1782 1608 4535]
 [   0    0    0 ... 2902  186 4512]
 [   0    0    0 ... 2302 4953 1099]
 ...
 [   0    0    0 ... 3204 4837  462]
 [   0    0    0 ... 1433 3096   15]
 [   0    0    0 ... 3443 2753 2533]]


In [31]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  830,
       4277, 4052,    9, 1874, 1493, 2622, 1782, 1608, 4535], dtype=int32)

In [32]:
## Creating model

# In normal LSTM layer what we have to do, we use model Sequential
embedding_vector_features=40
# The embedding_vector_features means all the particular index we got after the
# one_hot encoading and embedding representation has been converted into vectors
# by using Embedding Layers.

# embedding_vector_features(it is also called it as number of dimension)
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3,random_state = 42)

**Model Training**

In [37]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 11s 38ms/step - loss: 0.3276 - accuracy: 0.8465 - val_loss: 0.1943 - val_accuracy: 0.9156
Epoch 2/10
200/200 [==============================] - 7s 35ms/step - loss: 0.1365 - accuracy: 0.9456 - val_loss: 0.2139 - val_accuracy: 0.9185
Epoch 3/10
200/200 [==============================] - 7s 35ms/step - loss: 0.0945 - accuracy: 0.9646 - val_loss: 0.2250 - val_accuracy: 0.9112
Epoch 4/10
200/200 [==============================] - 8s 39ms/step - loss: 0.0645 - accuracy: 0.9776 - val_loss: 0.2833 - val_accuracy: 0.9125
Epoch 5/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0450 - accuracy: 0.9859 - val_loss: 0.3671 - val_accuracy: 0.9149
Epoch 6/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0293 - accuracy: 0.9903 - val_loss: 0.3194 - val_accuracy: 0.9129
Epoch 7/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0177 - accuracy: 0.9945 - val_loss: 0.5875 - val_accuracy: 0.906

**Adding Dropout**

In [38]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [39]:
y_pred = (model.predict(X_test) > 0.5)

In [45]:
from sklearn.metrics import confusion_matrix

In [46]:
confusion_matrix(y_test,y_pred)

array([[1481, 1626],
       [ 356, 2023]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6387167335034634

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.48      0.60      3107
           1       0.55      0.85      0.67      2379

    accuracy                           0.64      5486
   macro avg       0.68      0.66      0.64      5486
weighted avg       0.70      0.64      0.63      5486

